In [1]:
from datetime import datetime
import requests
import sys
import numpy as np
import pandas as pd

sys.path.append('..')
import peepop

from bokeh.io import output_notebook, show, push_notebook
output_notebook()

Loading BokehJS ...

In [2]:
from peepop.data.providers import Binance

api_key = 'JUY2Z2R1Lwpgta6toGtB8ofCxpiSBEiqqHJM8rp4GCWzIy2kXFbERuvxB2dDD1Cj'
secret_key = 'hDv51FaiLDHEA7Al2zhuG9YlXoKmgntjvgZDofFKNis1hY27x4WXHoL1RIsf1e8F'

binance = Binance(api_key, secret_key)

In [3]:
coin = peepop.instruments.Crypto('DOGE', 'USDT', provider = binance, interval = '30-min', start_date = '2021-05-01 00:00:00')

All caught up..!


In [4]:
# btc.plot_candles()

In [5]:
from peepop.strategy import TradingStrategy
from peepop.indicators import Indicator, SimpleMovingAverage, ExponentialMovingAverage, BollingerBands

class MACDCrossover(TradingStrategy):
    def setup(self):
        self.macd = peepop.indicators.MACD(fastperiod = 6, slowperiod = 18, signalperiod = 5, plot_separately = True)
        
        self.indicators = [self.macd]
    
    def logic(self, account, lookback):
        try:
            today = lookback.iloc[-1]
            
            macd_today, signal_today, _ = self.macd.lookback[-1]
            macd_yest, signal_yest, _ = self.macd.lookback[-2]
            
            # Buying
            buy_signal = (macd_today > signal_today) and (macd_yest < signal_yest)
            if buy_signal:
                entry_price   = today.close
                entry_capital = account.buying_power
                account.buy(entry_capital=entry_capital, entry_price=entry_price)
                
            # Selling
            sell_signal = (macd_today < signal_today) and (macd_yest > signal_yest)
            
            if sell_signal:
                percent = 1
                exit_price = today.close 
                account.sell(percent=percent, current_price=exit_price)
                
        except Exception as e:
#             print('ERROR', e)
            pass

In [7]:
strat = MACDCrossover(coin)

In [8]:
strat.backtest(start_date = '2021-06-1 00:00:00', commision = 0.001)

Performing backtest from: 01 June, 2021 (00:00:00) to 18 June, 2021 (11:30:00)



-------------- Results ----------------

Relative Returns: -18.09%
Relative Profit: -180.93

Strategy     : -25.92%
Net Profit   : -259.22

Buy and Hold : -7.83%
Net Profit   : -78.29

Buys        : 63
Sells        : 63
--------------------
Total Trades : 126

---------------------------------------


In [12]:
coindcx_api_key = 'fcb1c800d0ee02c93ea14bf61517afa0194d5c695eec9792'
coindcx_api_secret = '3071e32a059b72505f7d261d95aacb1ba66ab8e4cf5e89214bf329b454196c4c'

coindcxaccount = peepop.brokers.CoinDCX(btc, coindcx_api_key, coindcx_api_secret)

In [13]:
strat.execute(trading_account = coindcxaccount)

All caught up..!
